# Train Model

In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
pd.set_option('display.max_columns', None)

In [3]:
final_dataset = pd.read_csv("./data/1finalDataset.csv")
final_dataset

,AGE_DIFF,ATP_POINTS_DIFF,ATP_RANK_DIFF,BEST_OF,DRAW_SIZE,ELO_DIFF,ELO_GRAD_LAST_100_DIFF,ELO_GRAD_LAST_10_DIFF,ELO_GRAD_LAST_200_DIFF,ELO_GRAD_LAST_25_DIFF,ELO_GRAD_LAST_3_DIFF,ELO_GRAD_LAST_50_DIFF,ELO_GRAD_LAST_5_DIFF,ELO_SURFACE_DIFF,H2H_DIFF,H2H_SURFACE_DIFF,HEIGHT_DIFF,N_GAMES_DIFF,P_1ST_IN_LAST_100_DIFF,P_1ST_IN_LAST_10_DIFF,P_1ST_IN_LAST_200_DIFF,P_1ST_IN_LAST_25_DIFF,P_1ST_IN_LAST_3_DIFF,P_1ST_IN_LAST_50_DIFF,P_1ST_IN_LAST_5_DIFF,P_1ST_WON_LAST_100_DIFF,P_1ST_WON_LAST_10_DIFF,P_1ST_WON_LAST_200_DIFF,P_1ST_WON_LAST_25_DIFF,P_1ST_WON_LAST_3_DIFF,P_1ST_WON_LAST_50_DIFF,P_1ST_WON_LAST_5_DIFF,P_2ND_WON_LAST_100_DIFF,P_2ND_WON_LAST_10_DIFF,P_2ND_WON_LAST_200_DIFF,P_2ND_WON_LAST_25_DIFF,P_2ND_WON_LAST_3_DIFF,P_2ND_WON_LAST_50_DIFF,P_2ND_WON_LAST_5_DIFF,P_ACE_LAST_100_DIFF,P_ACE_LAST_10_DIFF,P_ACE_LAST_200_DIFF,P_ACE_LAST_25_DIFF,P_ACE_LAST_3_DIFF,P_ACE_LAST_50_DIFF,P_ACE_LAST_5_DIFF,P_BP_SAVED_LAST_100_DIFF,P_BP_SAVED_LAST_10_DIFF,P_BP_SAVED_LAST_200_DIFF,P_BP_SAVED_LAST_25_DIFF,P_BP_SAVED_LAST_3_DIFF,P_BP_SAVED_LAST_50_DIFF,P_BP_SAVED_LAST_5_DIFF,P_DF_LAST_100_DIFF,P_DF_LAST_10_DIFF,P_DF_LAST_200_DIFF,P_DF_LAST_25_DIFF,P_DF_LAST_3_DIFF,P_DF_LAST_50_DIFF,P_DF_LAST_5_DIFF,WIN_LAST_100_DIFF,WIN_LAST_10_DIFF,WIN_LAST_200_DIFF,WIN_LAST_25_DIFF,WIN_LAST_3_DIFF,WIN_LAST_50_DIFF,WIN_LAST_5_DIFF,RESULT
0,-5.0,-1028.0,69.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-5.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
1,-10.3,257.0,-126.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-8.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1
2,-3.7,-352.0,135.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
3,2.0,19.0,-7.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,7.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1
4,-0.9,-734.0,162.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-11.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Split Training vs Testing Data

In [4]:
# Convert data to numpy (exclude the first 5k matches, since ELO hasn't been properly calculated yet)
data = final_dataset.to_numpy(dtype=object)[5000:,:]
np.random.shuffle(data)

# Split the data using an 85% split between training and testing
split = 0.85
value = round(split*len(data))

data_train = data[:value,:]
data_test = data[value:,:]

print("Training Data: "+str(data_train.shape))
print("Testing Data: "+str(data_test.shape))

Training Data: (76819, 68)
Testing Data: (13556, 68)


In [5]:
# Define several mappers
mapper = np.vectorize(lambda x: "Player 2 Wins" if x == 0 else "Player 1 Wins")
reverse_mapper = np.vectorize(lambda x: 0 if x == "Player 2 Wins" else 1)

# Training data
x_train = data_train[:,:-1]
y_pred_train = mapper(data_train[:,-1:]).squeeze()

# Testing data
x_test = data_test[:,:-1]
y_pred_test = mapper(data_test[:,-1:]).squeeze()

## Train Models

### Train Simple Decision Tree

In [10]:
# Instantiate a Decision Tree
decision_sklearn = DecisionTreeClassifier(max_depth=4)
decision_sklearn = decision_sklearn.fit(x_train, y_pred_train)
joblib.dump(decision_sklearn, './models/decisiontree_mxdp4.joblib')

# Make predictions and test accuracy
predictions_train = decision_sklearn.predict(x_train)
predictions_test = decision_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.6563480388966272
Test Accuracy: 0.6637651224550015


In [9]:
text_representation = tree.export_text(decision_sklearn, feature_names=final_dataset.columns[:-1])
print(text_representation)

|--- ELO_DIFF <= 4.65
|   |--- ELO_DIFF <= -110.91
|   |   |--- ELO_DIFF <= -253.99
|   |   |   |--- ELO_DIFF <= -349.83
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ELO_DIFF >  -349.83
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_DIFF >  -253.99
|   |   |   |--- ELO_SURFACE_DIFF <= -140.15
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ELO_SURFACE_DIFF >  -140.15
|   |   |   |   |--- class: Player 2 Wins
|   |--- ELO_DIFF >  -110.91
|   |   |--- ELO_SURFACE_DIFF <= -28.28
|   |   |   |--- ATP_RANK_DIFF <= 129.50
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ATP_RANK_DIFF >  129.50
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_SURFACE_DIFF >  -28.28
|   |   |   |--- ATP_RANK_DIFF <= 209.50
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ATP_RANK_DIFF >  209.50
|   |   |   |   |--- class: Player 2 Wins
|--- ELO_DIFF >  4.65
|   |--- ELO_DIFF <= 150.19
|   |   |--- ELO_SURFACE_DIFF <= 45.05
|   |   |   |--- ATP_

### Train Random Forest

In [ ]:
# Instantiate a Random Forsest
forest_sklearn = RandomForestClassifier(n_estimators=500, max_depth=10, max_features="sqrt", bootstrap=True)
forest_sklearn = forest_sklearn.fit(x_train, y_pred_train)
joblib.dump(forest_sklearn, "./models/randomforest_est500_mxdp10.joblib")

# Make predictions and test accuracy
predictions_train = forest_sklearn.predict(x_train)
predictions_test = forest_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.7110350303961259
Test Accuracy: 0.6722484508704633


In [12]:
# Instantiate a Random Forsest
forest_sklearn2 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_split=400, min_samples_leaf=250, max_features="sqrt", bootstrap=True)
forest_sklearn2 = forest_sklearn2.fit(x_train, y_pred_train)
joblib.dump(forest_sklearn, "./models/randomforest_est100_mxdp7.joblib")

# Make predictions and test accuracy
predictions_train = forest_sklearn2.predict(x_train)
predictions_test = forest_sklearn2.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.6702508493992372
Test Accuracy: 0.6711419297727943


In [ ]:
# This is going to take a long time, if you want to comment it out (I already did this myself and the results are in the models folder)
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=5, 
    n_jobs=-1,
    verbose=4
)
grid_search.fit(x_train, y_pred_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100;, score=0.667 total time=  20.3s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100;, score=0.667 total time=  20.6s
[CV 3/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100;, score=0.663 total time=  20.6s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100;, score=0.661 total time=  21.1s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100;, score=0.669 total time=  21.0s
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=20, n_estimators=100;, score=0.666 total time=  21.7s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=5, min_samples_split=20, n_estimators=100;, score=0.66

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 20],
                         'n_estimators': [100, 300]},
             verbose=4)

In [14]:
# Best parameters
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

# Instantiate a Random Forsest
best_forest_model = RandomForestClassifier(max_depth=15, max_features='log2', min_samples_leaf=5, min_samples_split=20, n_estimators=300)
best_forest_model = best_forest_model.fit(x_train, y_pred_train)
joblib.dump(forest_sklearn, "./models/randomforestgridsearch_est300_mxdp15.joblib")

# Make predictions and test accuracy
predictions_train = best_forest_model.predict(x_train)
predictions_test = best_forest_model.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.8001796430570562
Test Accuracy: 0.6701829448214812


The gridSearchCV wasn't that successful, since the test accuracy was actually 0.6611. Let's train an XGBoost model and see if it does better.

### Train XGBoost Algorithm

Let's try with XGBoost and see if we can get better results.

In [18]:
# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, subsample=0.8, colsample_bytree=0.7)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

Train Accuracy: 0.9742381442091149
Test Accuracy: 0.6513720861611094


In [19]:
# Sorting by importance in descending order
sorted_mapped_results = sorted(
    list(zip(final_dataset.columns[:-1], list(xgb_model.feature_importances_))),
    key=lambda x: x[1], 
    reverse=True
)

# Extracting sorted labels and their importances
sorted_labels = [label for label, importance in sorted_mapped_results]
sorted_importances = [importance for _, importance in sorted_mapped_results]

# Displaying results
for label, importance in sorted_mapped_results:
    print(f"{label}: {importance:.4f}")


ELO_DIFF: 0.0843
ELO_SURFACE_DIFF: 0.0429
BEST_OF: 0.0204
ATP_RANK_DIFF: 0.0189
AGE_DIFF: 0.0161
WIN_LAST_25_DIFF: 0.0155
P_2ND_WON_LAST_100_DIFF: 0.0154
P_1ST_WON_LAST_10_DIFF: 0.0142
ATP_POINTS_DIFF: 0.0139
P_ACE_LAST_5_DIFF: 0.0138
P_1ST_WON_LAST_3_DIFF: 0.0138
ELO_GRAD_LAST_200_DIFF: 0.0137
H2H_SURFACE_DIFF: 0.0137
P_1ST_WON_LAST_25_DIFF: 0.0136
P_2ND_WON_LAST_200_DIFF: 0.0136
ELO_GRAD_LAST_50_DIFF: 0.0136
P_ACE_LAST_50_DIFF: 0.0136
ELO_GRAD_LAST_100_DIFF: 0.0135
H2H_DIFF: 0.0135
WIN_LAST_200_DIFF: 0.0135
P_1ST_WON_LAST_5_DIFF: 0.0135
P_2ND_WON_LAST_25_DIFF: 0.0135
P_2ND_WON_LAST_10_DIFF: 0.0135
P_DF_LAST_200_DIFF: 0.0135
P_1ST_WON_LAST_50_DIFF: 0.0135
P_1ST_IN_LAST_3_DIFF: 0.0134
P_1ST_IN_LAST_50_DIFF: 0.0134
P_ACE_LAST_25_DIFF: 0.0134
P_1ST_WON_LAST_100_DIFF: 0.0134
P_BP_SAVED_LAST_50_DIFF: 0.0133
P_2ND_WON_LAST_3_DIFF: 0.0133
N_GAMES_DIFF: 0.0133
P_DF_LAST_50_DIFF: 0.0133
P_ACE_LAST_200_DIFF: 0.0132
P_1ST_IN_LAST_200_DIFF: 0.0132
P_DF_LAST_25_DIFF: 0.0132
P_BP_SAVED_LAST_100_DIF

Okay this is overfitting significantly, let's try regularization.

In [21]:
# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.05, 
    subsample=0.7, 
    colsample_bytree=0.6,
    reg_alpha=0.1,
    reg_lambda=1.0
)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

Train Accuracy: 0.681524102110155
Test Accuracy: 0.6690764237238124


In [22]:
xgb_model.save_model("./models/xgb_model.json")

That's slightly better. Let's run a gridsearch to really make sure.

In [25]:
# Define parameter grid with all specified parameters
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.7],
    'colsample_bytree': [0.6],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.5, 1.0]
}

# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier()

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=3, 
    n_jobs=-1
)
grid_search.fit(x_train, reverse_mapper(y_pred_train))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 3/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=1.0, subsample=0.7;, score=0.665 total time=   2.9s
[CV 5/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=0.5, subsample=0.7;, score=0.670 total time=   3.0s
[CV 4/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=0.5, subsample=0.7;, score=0.665 total time=   3.0s
[CV 1/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=0.5, subsample=0.7;, score=0.668 total time=   3.0s
[CV 2/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=0.5, subsample=0.7;, score=0.672 total time=   3.1s
[CV 2/5] END colsample_bytree=0.6, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0.1, reg_lambda=1.0, s

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6],
                         'learning_rate': [0.01, 0.05], 'max_depth': [5, 10],
                         'n_estimators': [100, 300], 'reg_alpha': [0.1, 0.5],
                         'reg_lambda': [0.5, 1.0], 'subsample': [0.7]},
             scoring='accuracy', verbose=3)

In [26]:
# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Train the best model
best_xgb_model = grid_search.best_estimator_

# Make predictions
predictions_train = best_xgb_model.predict(x_train)
predictions_test = best_xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy:", accuracy_score(reverse_mapper(y_pred_train), predictions_train))
print("Test Accuracy:", accuracy_score(reverse_mapper(y_pred_test), predictions_test))

Best Parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 0.7}
Train Accuracy: 0.6818365248180789
Test Accuracy: 0.6672322218943642


In [27]:
best_xgb_model = grid_search.best_estimator_
best_xgb_model.save_model("./models/best_xgb_model.json")

### Train Neural Net

In [16]:
# Normalize the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Define the neural network
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train_scaled, reverse_mapper(y_pred_train), epochs=50, batch_size=32, validation_split=0.2, verbose=1)
model.save('./models/sequentialneuralnet64relx32relx16relx1sig.keras')
# Evaluate on test set
train_loss, train_acc = model.evaluate(x_train_scaled, reverse_mapper(y_pred_train), verbose=0)
test_loss, test_acc = model.evaluate(X_test_scaled, reverse_mapper(y_pred_test), verbose=0)

print(f"Train Accuracy: {train_acc}")
print(f"Test Accuracy: {test_acc}")

Epoch 1/50


d:\Github\TennisPredict\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1921/1921 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6492 - loss: 0.6215 - val_accuracy: 0.6641 - val_loss: 0.6013
Epoch 2/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6708 - loss: 0.6031 - val_accuracy: 0.6681 - val_loss: 0.6036
Epoch 3/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6763 - loss: 0.5979 - val_accuracy: 0.6671 - val_loss: 0.6044
Epoch 4/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6715 - loss: 0.5987 - val_accuracy: 0.6651 - val_loss: 0.6055
Epoch 5/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6772 - loss: 0.5972 - val_accuracy: 0.6651 - val_loss: 0.6009
Epoch 6/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6758 - loss: 0.5975 - val_accuracy: 0.6623 - val_loss: 0.6029
Epoch 7/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6819 - loss: 0.5924 - val_accuracy: 0.6641 - val_loss: 0.6027
Epoch 8/50
1921/1921 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6871 - loss: 0.5882 - val_accurac

The neural net wasn't the best result. I could try to optimize it, but since the video was mostly about random forest/decision trees, I focused more on that.

I expored the best result from the GridSearch and now we can use it to predict stuff :).

See the next notebook (3.Predict.ipynb) for this.